In [18]:
#Now we combine the three data sets of players, teams, and mvp data into one to use for the machine learning model
#Date cleaning portion 
import pandas as pd

In [19]:
mvps = pd.read_csv("mvps.csv")
# getting rid of redundant data that is available in other data sets
mvps = mvps[["Player", "Year", "Pts Won", "Pts Max", "Share"]]
mvps.head()

,Player,Year,Pts Won,Pts Max,Share
0,Michael Jordan,1991,891.0,960,0.928
1,Magic Johnson,1991,497.0,960,0.518
2,David Robinson,1991,476.0,960,0.496
3,Charles Barkley,1991,222.0,960,0.231
4,Karl Malone,1991,142.0,960,0.148


In [20]:
players = pd.read_csv("players.csv")
#Clean up the data by deleting certain rows
del players["Unnamed: 0"]
del players["Rk"]
players.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,Alaa Abdelnaby,PF,22,POR,43,0,6.7,1.3,2.7,.474,...,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,1991
1,Mahmoud Abdul-Rauf,PG,21,DEN,67,19,22.5,6.2,15.1,.413,...,0.5,1.3,1.8,3.1,0.8,0.1,1.6,2.2,14.1,1991
2,Mark Acres,C,28,ORL,68,0,19.3,1.6,3.1,.509,...,2.1,3.2,5.3,0.4,0.4,0.4,0.6,3.2,4.2,1991
3,Michael Adams,PG,28,DEN,66,66,35.5,8.5,21.5,.394,...,0.9,3.0,3.9,10.5,2.2,0.1,3.6,2.5,26.5,1991
4,Mark Aguirre,SF,31,DET,78,13,25.7,5.4,11.7,.462,...,1.7,3.1,4.8,1.8,0.6,0.3,1.6,2.7,14.2,1991


In [21]:
#fixing the names of the players to merge the players data with the mvps data
players["Player"].head(50)

0         Alaa Abdelnaby
1     Mahmoud Abdul-Rauf
2             Mark Acres
3          Michael Adams
4           Mark Aguirre
5            Danny Ainge
6            Mark Alarie
7           Steve Alford
8          Greg Anderson
9          Greg Anderson
10         Greg Anderson
11         Greg Anderson
12         Nick Anderson
13          Ron Anderson
14       Willie Anderson
15        Michael Ansley
16        B.J. Armstrong
17         Vincent Askew
18          Keith Askins
19           Miloš Babić
20          Thurl Bailey
21           Cedric Ball
22         Ken Bannister
23      Charles Barkley*
24           Dana Barros
25           John Battle
26          Kenny Battle
27          Kenny Battle
28          Kenny Battle
29       William Bedford
30       Benoit Benjamin
31       Benoit Benjamin
32       Benoit Benjamin
33       Winston Bennett
34           Larry Bird*
35      Rolando Blackman
36          Lance Blanks
37       Mookie Blaylock
38         Muggsy Bogues
39            Manute Bol


In [22]:
players["Player"] = players["Player"].str.replace("*", "", regex=False)

In [23]:
#creates a group of a player if they have multiple stats for different teams
players.groupby(["Player", "Year"])

In [24]:
#checks if the group has one row or multiple rows, if there are multiple rows, then we only get the total stats and set the team as the last team the player has played for
def single_row(df):
    if df.shape[0]==1:
        return df
    else:
        row = df[df["Tm"]=="TOT"]
        row["Tm"] = df.iloc[-1,:]["Tm"]
        return row
# split into groups and combined it again to get a multi-level index
players = players.groupby(["Player", "Year"]).apply(single_row)

In [25]:

players[players["Player"] == "Greg Anderson"]

Player Pos Age   Tm   G  GS    MP   FG   FGA  \
Player        Year                                                             
Greg Anderson 1991 8     Greg Anderson  PF  26  DEN  68   2  13.6  1.7   4.0   
              1992 467   Greg Anderson  PF  27  DEN  82  82  34.1  4.7  10.4   
              1994 1412  Greg Anderson  PF  29  DET  77  47  21.1  2.6   4.8   
              1995 1911  Greg Anderson  PF  30  ATL  51   0  12.2  1.1   2.0   
              1996 2381  Greg Anderson  PF  31  SAS  46   7   7.5  0.5   1.0   
              1997 2948  Greg Anderson   C  32  SAS  82  48  20.2  1.6   3.2   
              1998 3541  Greg Anderson   C  33  ATL  50   0   8.0  0.7   1.6   

                          FG%  ...  ORB  DRB   TRB  AST  STL  BLK  TOV   PF  \
Player        Year             ...                                            
Greg Anderson 1991 8     .430  ...  1.4  3.3   4.7  0.2  0.5  0.7  1.2  2.1   
              1992 467   .456  ...  4.1  7.4  11.5  1.0  1.1  0.8  2.5  3.2   
              1994 1412  .543  ...  2.4  5.0   7.4  0.7  0.7  0.9  1.2  3.0   
              1995 1911  .548  ...  1.2  2.5   3.7  0.3  0.5  0.6  0.6  2.0   
              1996 2381  .511  ...  0.6  1.5   2.2  0.2  0.2  0.5  0.5  1.4   
              1997 2948  .496  ...  1.9  3.5   5.5  0.4  0.8  0.8  0.9  2.7   
              1998 3541  .444  ...  0.8  1.6   2.4  0.3  0.4  0.2  0.3  1.7   

                          PTS  Year  
Player        Year                   
Greg Anderson 1991 8      4.3  1991  
              1992 467   11.5  1992  
              1994 1412   6.4  1994  
              1995 1911   2.9  1995  
              1996 2381   1.2  1996  
              1997 2948   3.9  1997  
              1998 3541   1.8  1998  

[7 rows x 30 columns]

In [26]:
#merge the two sets of data
combined = players.merge(mvps, how="outer", on=["Player", "Year"])
combined.head(5)

ValueError: 'Player' is both an index level and a column label, which is ambiguous.

In [ ]:
combined[combined["Pts Won"] > 0]

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,AST,STL,BLK,TOV,PF,PTS,Year,Pts Won,Pts Max,Share
197,Al Jefferson,C,29,CHA,73,73,35.0,9.6,18.8,.509,...,2.1,0.9,1.1,1.7,2.4,21.8,2014,34.0,1250.0,0.027
346,Allen Iverson,PG,21,PHI,76,74,40.1,8.2,19.8,.416,...,7.5,2.1,0.3,4.4,3.1,23.5,1997,1.0,1150.0,0.001
348,Allen Iverson,SG,23,PHI,48,48,41.5,9.1,22.0,.412,...,4.6,2.3,0.1,3.5,2.0,26.8,1999,319.0,1180.0,0.270
349,Allen Iverson,SG,24,PHI,70,70,40.8,10.4,24.8,.421,...,4.7,2.1,0.1,3.3,2.3,28.4,2000,132.0,1210.0,0.109
350,Allen Iverson,SG,25,PHI,71,71,42.0,10.7,25.5,.420,...,4.6,2.5,0.3,3.3,2.1,31.1,2001,1121.0,1240.0,0.904
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14170,Vince Carter,SF,23,TOR,82,82,38.1,9.6,20.7,.465,...,3.9,1.3,1.1,2.2,3.2,25.7,2000,51.0,1210.0,0.042
14171,Vince Carter,SF,24,TOR,75,75,39.7,10.2,22.1,.460,...,3.9,1.5,1.1,2.2,2.7,27.6,2001,7.0,1240.0,0.006
14175,Vince Carter,SF-SG,28,NJN,77,76,36.7,9.0,20.0,.452,...,4.2,1.4,0.6,2.2,3.2,24.5,2005,3.0,1270.0,0.002
14549,Yao Ming,C,23,HOU,82,82,32.8,6.5,12.5,.522,...,1.5,0.3,1.9,2.5,3.3,17.5,2004,1.0,1230.0,0.001


In [ ]:
#replacing Null columns with 
combined[["Pts Won", "Pts Max", "Share"]] = combined[["Pts Won", "Pts Max", "Share"]].fillna(0)

NameError: name 'combined' is not defined

In [ ]:
teams = pd.read_csv("teams.csv")
teams.head(30)

NameError: name 'pd' is not defined

In [ ]:
#gettng rid of rows with "division" in them and the stars in the team column
teams = teams[~teams["W"].str.contains("Division")]
teams["Team"] = teams["Team"].str.replace("*", "", regex=False)


,Unnamed: 0,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,0,51,31,.622,—,111.8,104.5,7.02,1991,Boston Celtics
1,1,51,31,.622,—,109.9,107.3,2.57,1991,Philadelphia 76ers
2,2,48,34,.585,3.0,109.4,107.1,2.38,1991,Toronto Raptors
3,3,44,38,.537,7.0,112.9,112.1,0.82,1991,Brooklyn Nets
4,4,37,45,.451,14.0,106.5,106.6,-0.01,1991,New York Knicks


In [ ]:
teams["Team"].unique

NameError: name 'teams' is not defined

In [ ]:
#Shorthand version of teams names so issue occurs when merging
combined["Tm"].unique().sort()

NameError: name 'combined' is not defined

In [ ]:
#We can use a csv file to map between the shorthand and the full name
import csv

with open('nicknames.csv', 'w+', newline='') as f:
    thewriter = csv.writer(f)

    thewriter.writerow(["Abbreviation", "Name"])
    thewriter.writerow(["ATL", "Atlanta Hawks"])
    thewriter.writerow(["BRK", "Brooklyn Nets"])
    thewriter.writerow(["BKN", "Brooklyn Nets"])
    thewriter.writerow(["BOS", "Boston Celtics"])
    thewriter.writerow(["CHA", "Charlotte Bobcats"])
    thewriter.writerow(["CHH", "Charlotte Hornets"])
    thewriter.writerow(["CHO", "Charlotte Hornets"])
    thewriter.writerow(["CHI", "Chicago Bulls"])
    thewriter.writerow(["CLE", "Cleveland Cavaliers"])
    thewriter.writerow(["DAL", "Dallas Mavericks"])
    thewriter.writerow(["DEN", "Denver Nuggets"])
    thewriter.writerow(["DET", "Detroit Pistons"])
    thewriter.writerow(["GSW", "Golden State Warriors"])
    thewriter.writerow(["HOU", "Houston Rockets"])
    thewriter.writerow(["IND", "Indiana Pacers"])
    thewriter.writerow(["LAC", "Los Angeles Clippers"])
    thewriter.writerow(["LAL", "Los Angeles Lakers"])
    thewriter.writerow(["MEM", "Memphis Grizzlies"])
    thewriter.writerow(["MIA", "Miami Heat"])
    thewriter.writerow(["MIL", "Milwaukee Bucks"])
    thewriter.writerow(["MIN", "Minnesota Timberwolves"])
    thewriter.writerow(["NJN", "New Jersey Nets"])
    thewriter.writerow(["NOH", "New Orleans Hornets"])
    thewriter.writerow(["NOP", "New Orleans Pelicans"])
    thewriter.writerow(["NOK", "New Orleans/Oklahoma City Hornets"])
    thewriter.writerow(["NYK", "New York Knicks"])
    thewriter.writerow(["NOH", "New Orleans Hornets"])
    thewriter.writerow(["OKC", "Oklamhoma City Thunder"])
    thewriter.writerow(["ORL", "Orlando Magic"])
    thewriter.writerow(["PHI", "Philadelphia 76ers"])
    thewriter.writerow(["PHX", "Phoenix Suns"])
    thewriter.writerow(["PHO", "Phoneix Suns"])
    thewriter.writerow(["POR", "Portland Trail Blazers"])
    thewriter.writerow(["SEA", "Seattle SuperSonics"])
    thewriter.writerow(["SAC", "Sacramento Kings"])
    thewriter.writerow(["SAS", "San Antonio Spurs"])
    thewriter.writerow(["TOR", "Toronto Raptors"])
    thewriter.writerow(["UTA", "Utah Jazz"])
    thewriter.writerow(["VAN", "Vancouver Grizzlies"])
    thewriter.writerow(["WAS", "Washington Wizards"])
    thewriter.writerow(["WSB", "Washington Bullets"])